<a href="https://colab.research.google.com/github/francianerod/PROJETOS-Empresas/blob/main/SoulCode_Assinatura_Tech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Considerações iniciais

## Equipe de Analistas

*   [Adriano Hoffman](https://www.linkedin.com/in/adriano-hoffmann-89104949/)
*   [Franciane Rodrigues](https://www.linkedin.com/in/francianerod/)
*   [Jonathas Carneiro](https://www.linkedin.com/in/sahtcarneiro/)
*   [Maria Lina S. J. Rocha](https://www.linkedin.com/in/lina-jeannine)
*   [Yuri Ageme](https://www.linkedin.com/in/yuri-ageme-ab2803167/)







## Observações

Os comentário do código serão separados entre comentário didáticos e comentários técnicos.

' ' '   
Assim estarão os comentários didáticos, na intenção de explicar decisões no decorrer do código e descrever o processo feito.   
' ' '   
  
' # Assim estarão os comentários técnicos que trarão ponto relevantes para a aplicação do script

#Setup

##Instalações

In [ ]:
pip install gcsfs

In [ ]:
!pip install firebase-admin

In [ ]:
pip install pyspark

In [ ]:
pip install pandera

In [ ]:
pip install pymongo

##Importações

In [ ]:
from google.colab import drive
from google.cloud import storage
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
from pymongo import MongoClient
import os
import json
import pandas as pd
import numpy as np
import pandera as pa
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, isnan, when, count
import warnings
warnings.filterwarnings("ignore")

##Conexões

In [ ]:
'''
Para cumprir os requisitos soliciatados precisariamos constantemente estar fazendos as conexões
com o firebase, bucket do cloud store e com o mongoDB Atlas, devido a isso já separamos as keys
de acesso no drive e automatizamos o processo de autenticação.
'''

###Drive

In [ ]:
#conexão com o drive onde consta as chaves necessários para acesso ao firebase,bucket gcp e mongoDB
#sem esse passo se fará necessário adicionar manualmente a chave ao scritp
drive.mount('/content/drive')
chave_firebase = '/content/drive/MyDrive/PROJETO FINAL/Chaves/firebase-dashboard-b2c-ad1-firebase-adminsdk-wju40-3a65f3c66f.json'
chave_bucket = '/content/drive/MyDrive/PROJETO FINAL/Chaves/bucket-dashboard-b2c-ad1-82fa16ca561e.json'
chave_mongo = '/content/drive/MyDrive/PROJETO FINAL/Chaves/mongo-X509-cert-2960257155044243010.pem'

###Firebase

In [ ]:
cred = credentials.Certificate(chave_firebase)
firebase_admin.initialize_app(cred)

###Bucket

In [ ]:
serviceAccount = chave_bucket
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [ ]:
client_bucket = storage.Client()
bucket = client_bucket.get_bucket('ad1-dashboards-b2c-tratados')

###MongoDB

In [ ]:
uri = "mongodb+srv://assinaturatech.hjwqgce.mongodb.net/?authSource=%24external&authMechanism=MONGODB-X509&retryWrites=true&w=majority"
client_mongo = MongoClient(uri,
                     tls=True,
                     tlsCertificateKeyFile=chave_mongo,
                     )

db = client_mongo['AssinaturaTech']


###PySpark


In [ ]:
spark = SparkSession.builder.appName("ass_tech").getOrCreate()

# ETL

In [ ]:
'''
De agora em diante começa todo o tratamento feito nos datasets disponibilizados.
Optamos por tratar cada dataset separadamente e, ao final, carregar todos uma única vez
para os destinos de armazenamento. Assim, garantimos os padrões de trabalho e normalização
ao mesmo tempo que mantemos a indiviadualidade de cada tratamento. Dessa meneira,
possibilitamos que fique clara qual a parte do script corresponde a cada um dos datasets
aqui existentes, caso eventualmente haja necessidade de trabalhar com algum individualmente.
'''

#Collection ShoppingCart

##Extração

In [ ]:
db = firestore.client()

docs = db.collection('shoppingCart').stream()

data = []
for doc in docs:
    data.append(doc.to_dict())

df_shoppingCart = pd.DataFrame(data)

In [ ]:
pd.set_option('display.max_columns', 200)

In [ ]:
df_shoppingCart.head(20)

##Tratamento

In [ ]:
df_shoppingCart.groupby('createdAt').size().sort_values(ascending=False)

###Remover colunas nulas

In [ ]:
colunas_nulas_shoppingCart = []
total_nulos_shoppingCart = 0

for c in list(df_shoppingCart.columns):
  if df_shoppingCart[c].count() == 0:
    colunas_nulas_shoppingCart.append(c)
    print(c)
    total_nulos_shoppingCart += 1
print(f'Total de colunas nulas: {total_nulos_shoppingCart}')

###Remover subcollections e colunas ilegiveis

In [ ]:
#Não foram identificadas colunas referentes a subcollections ou com caracteres ilegiveis na collection ShoppingCart

### Definição da Região do Cliente com base no DDD

In [ ]:
colunas_shoppingCart = list(df_shoppingCart.columns)

In [ ]:
for c in list(colunas_shoppingCart):
  print(f'coluna {c}: {list(df_shoppingCart[c].unique())}')

In [ ]:
df_shoppingCart['phoneNumber'] = df_shoppingCart['phoneNumber'].astype(str)

In [ ]:
df_shoppingCart['ddd'] = df_shoppingCart['phoneNumber'].apply(lambda x: x[1:3])

In [ ]:
df_shoppingCart['ddd'] = df_shoppingCart['ddd'].apply(lambda x: 0 if x=='on' else int(x))

In [ ]:
df_shoppingCart['createdAt'] = df_shoppingCart['createdAt'].astype(str)

In [ ]:
df_shoppingCart['hora'] = df_shoppingCart['createdAt'].apply(lambda x: x[11:13])

In [ ]:
df_shoppingCart['createdAt'] = df_shoppingCart['createdAt'].apply(pd.to_datetime)

In [ ]:
df_shoppingCart['data'] = df_shoppingCart['createdAt'].dt.strftime('%Y-%m-%d').str.slice(stop=10)

In [ ]:
ddd_estados = {
    11: 'São Paulo, SP, Brasil', 12: 'São José dos Campos, SP, Brasil', 13: 'Santos, SP, Brasil', 14: 'Baurú, SP, Brasil', 15: 'Sorocaba, SP, Brasil', 16: 'Ribeirão Preto, SP, Brasil', 17: 'São José do Rio Preto, SP, Brasil', 18: 'Presidente Prudente, SP, Brasil', 19: '	Campinas, SP, Brasil',
    21: 'Rio de Janeiro, RJ, Brasil', 22: 'Campos, RJ, Brasil', 24: 'Volta Redonda, RJ, Brasil',
    27: 'Vitória, ES, Brasil', 28: 'Cachoeiro de Itapemirim, ES, Brasil',
    31: 'Belo Horizonte, MG, Brasil', 32: 'Juiz de Fora, MG, Brasil', 33: 'Governador Valadares, MG, Brasil', 34: 'Uberlândia, MG, Brasil', 35: 'Varginha, MG, Brasil', 37: 'Divinópolis, MG, Brasil', 38: 'Montes Claros, MG, Brasil',
    41: 'Curitiba, PR, Brasil', 42: 'Ponta Grossa, PR, Brasil', 43: 'Londrina, PR, Brasil', 44: 'Maringá, PR, Brasil', 45: 'Cascável, PR, Brasil', 46: 'Pato Branco, PR, Brasil',
    47: 'Joinville, SC, Brasil', 48: 'Florianópolis, SC, Brasil', 49: 'Chapecó, SC, Brasil',
    51: 'Porto Alegre, RS, Brasil', 53: 'Pelotas, RS, Brasil', 54: 'Caxias do Sul, RS, Brasil', 55: 'Santa Maria, RS, Brasil',
    61: 'Brasília, DF, Brasil',
    62: 'Goiânia, GO, Brasil', 64: 'Rio Verde, GO, Brasil',
    63: 'Palmas, TO, Brasil',
    65: 'Cuiabá, MT, Brasil', 66: 'Rondonópolis, MT, Brasil',
    67: 'Campo Grande, MS, Brasil',
    68: 'Rio Branco, AC, Brasil',
    69: 'Porto Velho, RO, Brasil',
    71: 'Salvador, BA, Brasil', 73: 'Ilhéus, BA, Brasil', 74: 'Juazeiro, BA, Brasil', 75: 'Feira de Santana, BA, Brasil', 77: 'Barreiras, BA, Brasil',
    79: 'Aracajú, SE, Brasil',
    81: 'Recife, PE, Brasil', 87: 'Petrolina, PE, Brasil',
    82: 'Maceió, AL, Brasil',
    83: 'João Pessoa, PB, Brasil',
    84: 'Natal, RN, Brasil',
    85: 'Fortaleza, CE, Brasil',
    86: 'Teresina, PI, Brasil',
    88: 'Juazeiro do Norte, CE, Brasil',
    89: 'Picos, PI, Brasil',
    91: 'Belém, PA, Brasil', 93: 'Santarém, PA, Brasil', 94: 'Marabá, PA, Brasil',
    92: 'Manaus, AM, Brasil',
    95: 'Boa Vista, RR, Brasil',
    96: 'Macapá, AP, Brasil',
    97: 'Coari, AM, Brasil',
    98: 'São Luis, MA, Brasil', 99: 'Imperatriz, MA, Brasil'
}

In [ ]:
df_shoppingCart['estado'] = df_shoppingCart['ddd'].apply(lambda x : None if (x==0) | (x in [6,7,9,23,56,59]) else ddd_estados[x])

In [ ]:
df_shoppingCart['estado']

###Identificação das colunas principais

In [ ]:
#selcionando colunas mais relevantes
colunas_principais_shoppingCart = ['id',
                                   'phoneNumber',
                                   'ddd',
                                   'email',
                                   'integrationStatus',
                                   'purchaseStatus',
                                   'hora',
                                   'data',
                                   'planName',
                                   'price',
                                   'paymentMethod',
                                   'subscription_id',
                                   'estado',
                                   'customer_id']

df_shoppingCart_final = df_shoppingCart[colunas_principais_shoppingCart]
df_shoppingCart_final.head(2)

###Conversão de Tipos

In [ ]:
df_shoppingCart_final.info()

In [ ]:
df_shoppingCart_final['data'] = df_shoppingCart_final['data'].apply(pd.to_datetime)

### Tradução

In [ ]:
#renomeando colunas
df_shoppingCart_final.rename(columns={'id' : 'shoppingCart_id',
                                   'email' : 'email',
                                   'integrationStatus' : 'status_integracao',
                                   'purchaseStatus' : 'status_compra',
                                   'createdAt' : 'criado_em',
                                   'planName' : 'nome_plano',
                                   'price' : 'preco',
                                   'paymentMethod' : 'forma_pagamento',
                                   'subscription_id' : 'assinatura_id',
                                   'phoneNumber' : 'telefone',
                                   'createdAt' : 'criado_em',
                                   'customer_id' : 'cliente_id'
                     },inplace = True)

In [ ]:
#renomeando dados categóricos de status_integracao
status_integracao = {'Waiting': 'AGUARDANDO', 'Processed': 'PROCESSADO', 'Error': 'ERRO', 'Duplicity': 'DUPLICIDADE'}
df_shoppingCart_final['status_integracao'] = df_shoppingCart_final['status_integracao'].replace(status_integracao)

In [ ]:
#renomeando dados categóricos de status_compra
status_compra = {'Abandoned': 'ABANDONADO', 'Success': 'SUCESSO', 'Ordered': 'PEDIDO REALIZADO', 'Failure': 'FALHA'}
df_shoppingCart_final['status_compra'] = df_shoppingCart_final['status_compra'].replace(status_compra)

In [ ]:
#renomeando dados categóricos de assinatura
nomes_planos = {'Assinatura Tech Social ou Aluno': 'Tech Social ou Aluno ', 'Assinatura Tech Social ou Aluno + Assinatura Mulher': 'Tech Social ou Aluno + Mulher', 'Assinatura Tech Geral': 'Tech Geral', 'Assinatura Tech Mulher': 'Tech Mulher'}
df_shoppingCart_final['nome_plano'] = df_shoppingCart_final['nome_plano'].replace(nomes_planos)

In [ ]:
df_shoppingCart_final['nome_plano'] = df_shoppingCart_final['nome_plano'].str.upper()

### Identificação e Remoção da equipe soulcode da base de clientes

In [ ]:
df_shoppingCart_final['email'].unique()

In [ ]:
'''
Foi verificado que alguns emails não seguem o padrão de email (contendo @)
com isso não é possível separar os dominío corretamente, assim os emails
que não estão corretamente forneceidos serão dropados abaixo, para então separar
o domínio e identificar a equipe de testes da soulcode
'''

In [ ]:
df_shoppingCart_final.drop(df_shoppingCart_final.loc[~df_shoppingCart_final['email'].str.contains('@')].index, inplace=True)

In [ ]:
df_shoppingCart_final['dominio'] = df_shoppingCart_final['email'].apply(lambda x: x.split('@')[1])

In [ ]:
df_shoppingCart_final['dominio'].unique()

In [ ]:
df_soulcode = df_shoppingCart_final.loc[(df_shoppingCart_final['dominio'] == 'soulcode.com') |
                                        (df_shoppingCart_final['dominio'] == 'teste.com') |
                                        (df_shoppingCart_final['dominio'] == 'teste.com.br')
                                         ]
df_soulcode

In [ ]:
df_soulcode.groupby('dominio').size()

In [ ]:
df_shoppingCart_final.info()

In [ ]:
for dominio in list(df_soulcode['dominio'].unique()):
  df_shoppingCart_final = df_shoppingCart_final.loc[df_shoppingCart_final['dominio'] != dominio]

In [ ]:
df_shoppingCart_final.info()

## Pré Análise e Insights Iniciais

In [ ]:
df_shoppingCart_final.head(20)

###Status dos pedidos

In [ ]:
df_shoppingCart_final.info()

In [ ]:
#Status dos pedidos por quantidade
df_shoppingCart_final.groupby('status_compra').size()

In [ ]:
##Status dos pedidos por percentual
df_shoppingCart_final['status_compra'].value_counts(normalize=True)*100

In [ ]:
df_shoppingCart_final['status_compra'].value_counts(normalize=True).plot.pie(autopct='%1.1f%%')

###Taxa de abandono de carrinho

In [ ]:
len(df_shoppingCart_final.loc[df_shoppingCart_final['status_compra']=='ABANDONADO'])/len(df_shoppingCart_final['shoppingCart_id'])*100

###Status pedidos x tipo de plano

In [ ]:
for item in list(df_shoppingCart_final['status_compra'].unique()):
  print(f'Status do Pedido: {item}')
  print(df_shoppingCart_final.loc[df_shoppingCart_final['status_compra']==item].groupby('nome_plano')['shoppingCart_id'].size())
  print('')

OBS: o total de pedidos por plano não é igual ao total de pedidos por status, o df tem 5378 linhas ao todo, mas a coluna planName possui apenas 1324 linhas non-null

###Status pedidos x status integração

In [ ]:
for item in list(df_shoppingCart_final['status_compra'].unique()):
  print(f'Status do Pedido: {item}')
  print(df_shoppingCart_final.loc[df_shoppingCart_final['status_compra']==item].groupby('status_integracao').size())
  print('')

###Status pedidos x Montante(R$)

In [ ]:
df_shoppingCart_final.groupby('status_compra')['preco'].sum()

In [ ]:
df_shoppingCart_final.groupby('status_compra')['preco'].sum().plot.bar()

## Conclusões da Pré Análise

- A maior parte dos pedidos está na categoria "Abandoned", indicando um possível problema com o abandono do carrinho (a taxa de abandono é de 67,9%)
- A maioria dos pedidos "Abandoned" e "Failure" foram feitos nos planos "Assinatura Tech Social ou Aluno" e "Assinatura Tech Social ou Aluno + Assinatura Mulher". Enquanto isso, a maioria dos pedidos "Success" foi feita no plano "Assinatura Tech Social ou Aluno".   
- A maioria dos pedidos "Abandoned" está no status "Waiting" de integração, o que pode indincar um possível problema no processo de integração com a plataforma de pagamento.   
- O montante total de pedidos "Abandoned" é significativamente maior do que os outros status de pedidos, o que nos indica que a diminuição da taxa de abandono de carrinho pode incorrer em uma diferença considerável no faturamento da empresa.

## Validação dos dados com o SCHEMA

In [ ]:
df_shoppingCart_final.info()

In [ ]:
schema_shoppingCart_final = pa.DataFrameSchema({
    "shoppingCart_id": pa.Column(pa.String),
    "email": pa.Column(pa.String),
    "status_integracao": pa.Column(pa.String),
    "status_compra": pa.Column(pa.String),
    "data": pa.Column(pa.DateTime),
    "nome_plano": pa.Column(pa.String, nullable=True),
    "preco": pa.Column(pa.Float, nullable=True),
    "forma_pagamento": pa.Column(pa.String, nullable=True),
    "assinatura_id": pa.Column(pa.String, nullable=True),
    "cliente_id": pa.Column(pa.String, nullable=True)
})

In [ ]:
df_carrinho = schema_shoppingCart_final.validate(df_shoppingCart_final)

#Collection Subscriptions

##Extração

In [ ]:
db = firestore.client()

docs = db.collection('subscriptions').stream()

data = []
for doc in docs:
    data.append(doc.to_dict())

df_subscriptions = pd.DataFrame(data)

In [ ]:
pd.set_option('display.max_columns', 200)

In [ ]:
df_subscriptions.head(20)

In [ ]:
df_subscriptions.info()

##Tratamento

###Remover colunas nulas

In [ ]:
colunas_nulas_subscriptions = []
total_nulos_subscriptions = 0

for c in list(df_subscriptions.columns):
  if df_subscriptions[c].count() == 0:
    colunas_nulas_subscriptions.append(c)
    print(c)
    total_nulos_subscriptions += 1
print(f'Total de colunas nulas: {total_nulos_subscriptions}')

In [ ]:
df_subscriptions.dropna(axis=1, how='all', inplace=True)

In [ ]:
df_subscriptions.info()

###Identificação das colunas principais

In [ ]:
df_subscriptions = df_subscriptions[['id',
                                    'customer_id',
                                    'customer_name',
                                    'customer_email',
                                    'created_at',
                                    'updated_at',
                                    'expires_at',
                                    'plan_identifier',
                                    'active',
                                    'suspended',
                                    'payable_with',
                                    'recent_invoices',
                                    'custom_variables',
                                    'subitems',
                                    'logs'
                                    ]]

In [ ]:
'''
Abaixo percebemos que para um mesmo customer_id haviam vários registros,
referentes às atualizações da assinatura. Optamos por ordená-los com base na data
de atualização e manter o último registro, preservando apenas a informação
da situação atual daquela assinatura.
'''

In [ ]:
df_subscriptions = df_subscriptions.sort_values(by=['updated_at'])
df_subscriptions = df_subscriptions.drop_duplicates(['customer_id'], keep='last')

In [ ]:
df_subscriptions

###Extração das subcollections

In [ ]:
#extração da coluna subitems
df_subitems = df_subscriptions['subitems'].apply(pd.Series)
df_subitems

In [ ]:
#extração da coluna custom_variables
df_custom_variables = df_subscriptions['custom_variables'].apply(pd.Series)
df_custom_variables

In [ ]:
#extração da coluna recent_invoices
df_recent_invoices = df_subscriptions['recent_invoices'].apply(pd.Series)
df_recent_invoices

#### Extrair informações sobre promoções e descontos de Subitems

In [ ]:
'''
PASSO: percorrer colunas do df_subitems em busca de valores para a chave description.
Cada celula do DF é um dicionário com chave:valor correspondente ao subitem que compõe um assinatura

Os subitems atuais podem ser:
1 - Ebooks
2 - Serviços
3 - Lote promocional
4 - Desconto
'''

In [ ]:
#Filtro para encontrar os valores quando a chave description for referente ao lote
def filtro_lote(x):
  for c in list(df_subitems.columns):
    if pd.isna(x[c]):
      continue
    elif x[c]['description'][:4] == 'Lote':
      return x[c]['description']

In [ ]:
#Criação da Coluna Lote no df_subitems
df_subitems['lote_promocional'] = df_subitems.apply(filtro_lote, axis=1)

In [ ]:
##Valores unicos para a variavel lote:
list(df_subitems['lote_promocional'].unique())

In [ ]:
##Para algumas assinaturas a composição de subitems não inclui o lote promocional, nesses casos a assinatura é composta de Ebooks + Serviços + Desconto
##nesse caso vamos substituir o None por sem promoção:
df_subitems['lote_promocional'] = df_subitems['lote_promocional'].apply(lambda x: 'Sem Promoção' if x is None else x)

In [ ]:
#Filtro para encontrar os valores quando a chave description for referente ao desconto
#como o df_subitems tem uma nova col (lote) que não é um dict, temos que deixá-la de fora do loop

def filtro_desconto(x):
  for c in range(4):
    if pd.isna(x[c]):
      continue
    elif x[c]['description'][:4] == 'Desc':
      return x[c]['description']

In [ ]:
#Criação da Coluna desconto no df_subitems
df_subitems['desconto'] = df_subitems.apply(filtro_desconto, axis=1)

In [ ]:
##Valores unicos para a variavel desconto:
list(df_subitems['desconto'].unique())

In [ ]:
df_subitems['desconto'] = df_subitems['desconto'].apply(lambda x: 'Sem Desconto' if x is None else x)

In [ ]:
def filtro_campanha(x):
  if x['lote_promocional'] == 'Sem Promoção' and x['desconto'] == 'Sem Desconto':
    return 'Padrão'
  elif x['lote_promocional'] != 'Sem Promoção' and x['desconto'] == 'Sem Desconto':
    return 'Lote Promocional'
  elif x['lote_promocional'] == 'Sem Promoção' and x['desconto'] != 'Sem Desconto':
    return 'Desconto'
  else:
    return 'Lote Promocional + Desconto'

In [ ]:
df_subitems['campanha'] = df_subitems.apply(filtro_campanha, axis=1)

In [ ]:
##Valores unicos para a variavel campanha:
list(df_subitems['campanha'].unique())

In [ ]:
df_subitems.info()

In [ ]:
# PASSO: Concatenar df_subscriptions e df_subitems em um único df
df_subscriptions_geral = pd.concat([df_subscriptions,df_subitems.drop(df_subitems.columns[[0,1,2,3]],axis=1)], axis=1)

In [ ]:
df_subscriptions_geral

#### Extrair informações de venda da Custom_variables

In [ ]:
df_custom_variables[0].apply(pd.Series)

In [ ]:
df_custom_variables[0].apply(pd.Series).pivot(columns='name', values='value')

In [ ]:
def filtro_venda(x):
  for c in range(len(x)):
    if x[c]['name'] == 'sale_type':
      return x[c]['value']

In [ ]:
df_custom_variables['tipo_venda'] = df_custom_variables.apply(filtro_venda, axis=1)

In [ ]:
df_custom_variables.info()

In [ ]:
df_custom_variables['tipo_venda'].unique()

In [ ]:
def filtro_canal(x):
  colunas = [0,2,3,4,5,6,7,8]
  for c in colunas:
    if pd.isna(x[c]):
      continue
    elif x[c]['name'] == 'channel':
      if x[c]['value'] == 'null':
        return None
      else:
        return x[c]['value']

In [ ]:
df_custom_variables['canal'] = df_custom_variables.apply(filtro_canal, axis=1)

In [ ]:
df_custom_variables.info()

In [ ]:
df_custom_variables['canal'].unique()

In [ ]:
df_custom_variables[['tipo_venda','canal']]

In [ ]:
# PASSO: Concatenar df_subscriptions_geral e df_custom_variables em um único df
df_subscriptions_geral = pd.concat([df_subscriptions_geral,df_custom_variables[['tipo_venda','canal']]], axis=1)

#### Extrair status da fatura mais recente de Recent_invoice

In [ ]:
df_recent_invoices

In [ ]:
df_recent_invoices[0].apply(pd.Series)

In [ ]:
def filtro_status_fatura(x):
  for c in list(df_recent_invoices.columns):
    if pd.isna(x[c]):
      continue
    else:
      return x[c]['status']

In [ ]:
df_recent_invoices['fatura_status'] = df_recent_invoices.apply(filtro_status_fatura, axis=1)

In [ ]:
list(df_recent_invoices['fatura_status'].unique())

In [ ]:
df_recent_invoices.info()

In [ ]:
df_subscriptions_geral = pd.concat([df_subscriptions_geral,df_recent_invoices[['fatura_status']]], axis=1)

###Remover subcollections

In [ ]:
#Retirando colunas
df_subscriptions_geral.drop(['recent_invoices',
                            'custom_variables',
                            'subitems',
                            'logs'],axis=1,inplace=True)

### Tradução

In [ ]:
df_subscriptions_geral

In [ ]:
#Traduzindo colunas
df_subscriptions_geral.rename(columns={'id': 'assinatura_id',
                             'customer_id': 'cliente_id',
                             'customer_name': 'cliente_nome',
                             'customer_email':'cliente_email',
                             'created_at': 'criado_em',
                             'updated_at': 'ultima_atualizacao',
                             'expires_at': 'expira_em',
                             'plan_identifier': 'tipo_de_plano',
                             'active': 'ativo',
                             'suspended': 'suspenso',
                             'payable_with': 'a_pagar_com',
                             'recent_invoices.status': 'fatura_status',
                             'subitems.description': 'subitem_descricao',
                             'logs.description': 'registro_descricao',
                            },inplace = True)

In [ ]:
#Traduzindo dados
df_subscriptions_geral.loc[df_subscriptions_geral.tipo_de_plano == 'assinatura_tech_social_recurring',   ['tipo_de_plano']] = 'ASSINATURA TECH SOCIAL RECORRENTE'
df_subscriptions_geral.loc[df_subscriptions_geral.tipo_de_plano == 'assinatura_tech_geral',              ['tipo_de_plano']] = 'ASSINATURA TECH GERAL'
df_subscriptions_geral.loc[df_subscriptions_geral.tipo_de_plano == 'assinatura_tech_geral_recurring',    ['tipo_de_plano']] = 'ASSINATURA TECH GERAL RECORRENTE'
df_subscriptions_geral.loc[df_subscriptions_geral.tipo_de_plano == 'assinatura_tech_social',             ['tipo_de_plano']] = 'ASSINATURA TECH SOCIAL'

df_subscriptions_geral.loc[df_subscriptions_geral.a_pagar_com == 'credit_card',   ['a_pagar_com']] = 'CARTÃO DE CRÉDITO'
df_subscriptions_geral.loc[df_subscriptions_geral.a_pagar_com == 'bank_slip',     ['a_pagar_com']] = 'BOLETO BANCÁRIO'
df_subscriptions_geral.loc[df_subscriptions_geral.a_pagar_com == 'pix',           ['a_pagar_com']] = 'PIX'

df_subscriptions_geral.loc[df_subscriptions_geral.fatura_status == 'canceled',               ['fatura_status']] = 'CANCELADO'
df_subscriptions_geral.loc[df_subscriptions_geral.fatura_status == 'paid',                   ['fatura_status']] = 'PAGO'
df_subscriptions_geral.loc[df_subscriptions_geral.fatura_status == 'expired',                ['fatura_status']] = 'EXPIRADO'
df_subscriptions_geral.loc[df_subscriptions_geral.fatura_status == 'refunded',               ['fatura_status']] = 'REEMBOLSADO'
df_subscriptions_geral.loc[df_subscriptions_geral.fatura_status == 'pending',                ['fatura_status']] = 'PENDENTE'
df_subscriptions_geral.loc[df_subscriptions_geral.fatura_status == 'partially_refunded',     ['fatura_status']] = 'PARCIALMENTE REEMBOLSADO'
df_subscriptions_geral.loc[df_subscriptions_geral.fatura_status == 'chargeback',             ['fatura_status']] = 'ESTORNO'
df_subscriptions_geral.loc[df_subscriptions_geral.fatura_status == 'in_protest',             ['fatura_status']] = 'EM PROTESTO'


df_subscriptions_geral['cliente_nome'] = df_subscriptions_geral['cliente_nome'].str.upper()
df_subscriptions_geral['lote_promocional'] = df_subscriptions_geral['lote_promocional'].str.upper()
df_subscriptions_geral['desconto'] = df_subscriptions_geral['desconto'].str.upper()
df_subscriptions_geral['campanha'] = df_subscriptions_geral['campanha'].str.upper()
df_subscriptions_geral['tipo_venda'] = df_subscriptions_geral['tipo_venda'].str.upper()
df_subscriptions_geral['canal'] = df_subscriptions_geral['canal'].str.upper()


In [ ]:
df_subscriptions_geral.info()

In [ ]:
df_subscriptions_geral['tipo_venda'] = df_subscriptions_geral['tipo_de_plano'].apply(lambda x: 'RECORRENTE' if 'RECORRENTE' in x else 'GARANTIDA')

In [ ]:
df_subscriptions_geral

### Identificação e Remoção da equipe soulcode da base de clientes

In [ ]:
df_subscriptions_geral['dominio'] = df_subscriptions_geral['cliente_email'].apply(lambda x: x.split('@')[1])

In [ ]:
df_subscriptions_geral['dominio'].unique()

In [ ]:
df_soulcode = df_subscriptions_geral.loc[(df_subscriptions_geral['dominio'] == 'soulcode.com') |
                                        (df_subscriptions_geral['dominio'] == 'teste.com') |
                                        (df_subscriptions_geral['dominio'] == 'teste.com.br')
                                         ]
df_soulcode

In [ ]:
df_soulcode.groupby('dominio').size()

In [ ]:
df_subscriptions_geral.info()

In [ ]:
for dominio in list(df_soulcode['dominio'].unique()):
  df_subscriptions_geral = df_subscriptions_geral.loc[df_subscriptions_geral['dominio'] != dominio]

In [ ]:
df_subscriptions_geral.info()

## Pré Análise e Insights Iniciais

In [ ]:
df_subscriptions_geral.head(20)

###Metodo de Pagamento mais utilizado

In [ ]:
df_subscriptions_geral.groupby('a_pagar_com').size()

In [ ]:
df_subscriptions_geral.groupby('a_pagar_com').size().plot.bar()

OBS: o número de assinutauras é 5.000 porém só temos 2.731 clientes únicos. Um cliente pode ter mais de uma assinatura?

###Metodo de Pagamento x tipo de plano

In [ ]:
df_subscriptions_geral.groupby(['a_pagar_com','tipo_de_plano']).size()

###Planos Ativos

In [ ]:
df_subscriptions_geral.groupby('ativo').size()

In [ ]:
df_subscriptions_geral['ativo'].value_counts(normalize=True)*100

In [ ]:
df_subscriptions_geral.groupby('ativo').size().plot.pie(autopct='%1.1f%%')

###Planos Ativos x Suspensos

In [ ]:
df_subscriptions_geral.groupby(['ativo','suspenso']).size()

In [ ]:
df_subscriptions_geral.loc[df_subscriptions_geral['ativo']==True]['suspenso'].value_counts(normalize=True)*100

In [ ]:
df_subscriptions_geral.groupby(['ativo','suspenso']).size().plot.pie(autopct='%1.1f%%')

OBS: Verificar o significado de 'suspenso'. Como pode um plano estar ativo e suspenso ao mesmo tempo?

###Prazo de validade dos planos

In [ ]:
df_subscriptions_geral.groupby('expira_em').size()

In [ ]:
df_subscriptions_geral.groupby('expira_em').size().plot.line()

OBS: Fatiar a data por mês e ano

## Conclusão da Pré Análise
- O método de pagamento mais utilizado nas assinaturas é o cartão de crédito, seguido pelo boleto bancário e Pix.  
- O plano "Assinatura Tech Social" é o mais adquirido com boleto e cartão de crédito, enquanto o plano "Assinatura Tech Geral" tem mais adesões com Pix.
71,7% dos planos estão ativos, enquanto 28,3% estão suspensos.   
- Dos planos que estão ativos, 58% estão com a mensalidade em dia (suspended = False) e 42% estão com atrasos ou não especificado (suspended = True).
- A maioria dos planos ativos tem prazo de validade em 2024, com uma grande concentração no mês de março de 2024.

## Validação dos dados com o SCHEMA

In [ ]:
df_subscriptions_geral.info()

In [ ]:
df_subscriptions_geral.replace(to_replace="None", value=np.NaN, inplace=True)

In [ ]:
df_subscriptions_geral["ativo"] = df_subscriptions_geral["ativo"].map({True: "SIM", False: "NAO"})
df_subscriptions_geral["suspenso"] = df_subscriptions_geral["suspenso"].map({True: "SIM", False: "NAO"})

In [ ]:
df_subscriptions_geral['criado_em'] = df_subscriptions_geral['criado_em'].astype('datetime64[ns]')
df_subscriptions_geral['ultima_atualizacao'] = df_subscriptions_geral['ultima_atualizacao'].astype('datetime64[ns]')

In [ ]:
schema_subscriptions_geral = pa.DataFrameSchema({
    "assinatura_id": pa.Column(pa.String, nullable=False),
    "cliente_id": pa.Column(pa.String, nullable=False),
    "cliente_nome": pa.Column(pa.String, nullable=False),
    "cliente_email": pa.Column(pa.String, nullable=False),
    "criado_em": pa.Column(pa.DateTime, nullable=False),
    "ultima_atualizacao": pa.Column(pa.DateTime, nullable=False),
    "expira_em": pa.Column(pa.String, nullable=False),
    "tipo_de_plano": pa.Column(pa.String, nullable=False),
    "ativo": pa.Column(pa.String, pa.Check.str_length(3,3), nullable=False),
    "suspenso": pa.Column(pa.String, pa.Check.str_length(3,3), nullable=False),
    "a_pagar_com": pa.Column(pa.String, nullable=False),
    "lote_promocional": pa.Column(pa.String, nullable=False),
    "desconto": pa.Column(pa.String, nullable=False),
    "campanha": pa.Column(pa.String, nullable=False),
    "tipo_venda": pa.Column(pa.String, nullable=False),
    "canal": pa.Column(pa.String, nullable=True),
    "fatura_status": pa.Column(pa.String, nullable=True),
    "dominio": pa.Column(pa.String, nullable=False),
})

In [ ]:
df_assinaturas = schema_subscriptions_geral.validate(df_subscriptions_geral)

#Collection Invoice

##Extração

In [ ]:
db = firestore.client()

docs = db.collection('invoices').stream()

data = []
for doc in docs:
    data.append(doc.to_dict())

df_invoices = pd.DataFrame(data)

In [ ]:
pd.set_option('display.max_columns', 200)

In [ ]:
df_invoices.head(20)

In [ ]:
df_invoices.info(max_cols=200)

##Tratamento

###Remover colunas nulas

In [ ]:
colunas_invoice = list(df_invoices.columns)

In [ ]:
#verificando existência de colunas nulas

colunas_nulas_invoice = []
total_nulos_invoice = 0

for c in colunas_invoice:
  if df_invoices[c].count() == 0:
    colunas_nulas_invoice.append(c)
    print(c)
    total_nulos_invoice += 1
print(f'Total de colunas nulas: {total_nulos_invoice}')

In [ ]:
for c in colunas_invoice:
  if c in colunas_nulas_invoice:
    colunas_invoice.remove(c)

print(len(colunas_invoice))

###Extração da subcollection Variables

In [ ]:
df_variables = df_invoices[['id','variables']]
df_variables

In [ ]:
variables_explode = df_variables.explode('variables')
df_variables = pd.json_normalize(json.loads(variables_explode.to_json(orient="records")))
df_variables

In [ ]:
df_variables.drop_duplicates(inplace=True)

In [ ]:
df_variables = df_variables.pivot(index='id', columns='variables.variable', values='variables.value')
df_variables.reset_index(inplace=True)
df_variables

In [ ]:
df_variables = df_variables[['id','subscription_id']]
df_variables

In [ ]:
df_variables.drop(columns=['id'],inplace=True)

In [ ]:
df_invoices = pd.concat([df_invoices, df_variables], axis=1)
df_invoices.head(20)

###Remover subcollections e colunas ilegiveis

In [ ]:
colunas_subcollection_invoice = ['bank_slip','credit_card_transaction', 'financial_return_dates', 'logs', 'pix', 'variables', 'items']
print(f'Qtd de Subcollections: {len(colunas_subcollection_invoice)}')

In [ ]:
for c in colunas_invoice:
  if c in colunas_subcollection_invoice:
    colunas_invoice.remove(c)

print(len(colunas_invoice))

In [ ]:
colunas_ilegiveis_invoice = ['early_payment_discounts','custom_variables']
print(f'Qtd de colunas ilegiveis: {len(colunas_ilegiveis_invoice)}')

In [ ]:
for c in colunas_invoice:
  if c in c in colunas_ilegiveis_invoice:
    colunas_invoice.remove(c)

print(len(colunas_invoice))

###Identificação das colunas principais

In [ ]:
'''for c in list(colunas_invoice[:20]):
  print(f'coluna {c}: {list(df_invoices[c].unique())}')'''
for c in list(colunas_invoice[:20]):
  unique_values = df_invoices[c].apply(lambda x: str(x) if isinstance(x, dict) else x).unique()
  print(f'coluna {c}: {list(unique_values)}')

In [ ]:
for c in list(colunas_invoice[20:40]):
  print(f'coluna {c}: {list(df_invoices[c].unique())}')

In [ ]:
for c in list(colunas_invoice[40:60]):
    try:
        unique_values = list(df_invoices[c].unique())
    except Exception as e:
        print(f'Erro na coluna {c}: {str(e)}')
        continue
    print(f'Coluna {c}: {unique_values}')

In [ ]:
for c in list(colunas_invoice[60:]):
  print(f'coluna {c}: {list(df_invoices[c].unique())}')

In [ ]:
colunas_invoices_principais = ['id',
                               'customer_id',
                               'subscription_id',
                               'email',
                               'status',
                               'paid_cents',
                               'discount_cents',
                               'late_payment_fine',
                               'taxes_paid_cents',
                               'total_paid',
                               'total_cents',
                               'items_total_cents',
                               'total_on_occurrence_day_cents',
                               'refunded_cents',
                               'total_refunded',
                               'installments',
                               'max_installments_value',
                               'total_overpaid',
                               'payable_with',
                               'payment_method',
                               'payer_phone_prefix',
                               'credit_card_brand',
                               'ignore_due_email',
                               'refundable',
                               'created_at_iso',
                               'updated_at',
                               'paid_at',
                               'expired_at_iso',
                               'protested_at_iso',
                               'canceled_at_iso',
                               'chargeback_at_iso',
                               'refunded_at',
                               'occurrence_date'
                               ]


In [ ]:
df_invoices_final = df_invoices[colunas_invoices_principais]
df_invoices_final.head(20)

###Tradução

In [ ]:
df_faturas = df_invoices_final.rename(columns={
    'id' : 'fatura_id',
    'customer_id': 'cliente_id',
    'subscription_id': 'assinatura_id',
    'paid_cents': 'pago_centavos',
    'discount_cents' : 'disconto_centavos',
    'late_payment_fine' : 'multa_pagamento_atrasado',
    'taxes_paid_cents' : 'imposto_pago_centavos',
    'total_cents':'total_centavos',
    'total_paid' : 'total_pago',
    'items_total_cents':'items_total_centavos',
    'total_on_occurrence_day_cents':'total_na_ocorrencia_centavos',
    'max_installments_value':'max_parcelas',
    'installments':'parcelas',
    'refunded_cents':'reembolsado_centavos',
    'total_refunded':'total_reembolsado',
    'total_overpaid':'total_pago_em_excesso',
    'payable_with':'a_pagar_com',
    'payment_method':'forma_pagamento',
    'payer_phone_prefix':'ddd',
    'credit_card_brand':'cartao_de_credito_marca',
    'ignore_due_email':'email_devido_ignorado',
    'refundable':'reembolsavel',
    'created_at_iso':'criado_em',
    'updated_at':'atualizado_em',
    'paid_at':'pago_em',
    'expired_at_iso':'expirado_em',
    'protested_at_iso':'protestado_em',
    'canceled_at_iso':'cancelado_em',
    'chargeback_at_iso':'chargeback_em',
    'refunded_at':'reembolsado_em',
    'occurrence_date':'data_ocorrencia'
})


In [ ]:
df_faturas.info()

In [ ]:
for c in df_faturas.columns:
  print(f'coluna {c}: {list(df_faturas[c].unique())}')

In [ ]:
traducoes = {
    'canceled' : 'CANCELADO',
    'paid' : 'PAGO',
    'expired' : 'EXPIRADO',
    'pending':'PENDENTE',
    'refunded':'REEMBOLSADO',
    'partially_refunded':'PARCIALMENTE REEMBOLSADO',
    'chargeback':'CHARGEBACK',
    'in_protest':'EM PROTESTO'

    }

for key, value in traducoes.items():
  df_faturas.loc[df_faturas.status==key,'status'] = value

In [ ]:
traducoes = {
    'credit_card' : 'CARTAO DE CREDITO',
    'pix' : 'PIX',
    'bank_slip' : 'BOLETO BANCARIO',
    'all':'TODOS'}

for key, value in traducoes.items():
  df_faturas.loc[df_faturas.a_pagar_com==key,'a_pagar_com'] = value

In [ ]:
traducoes = {
    'master' : 'MASTER',
    'visa' : 'VISA',
    'elo' : 'ELO',
    'amex':'AMEX'}

for key, value in traducoes.items():
  df_faturas.loc[df_faturas.cartao_de_credito_marca==key,'cartao_de_credito_marca'] = value

In [ ]:
traducoes = {
    'iugu_credit_card' : 'CARTAO DE CREDITO',
    'iugu_pix' : 'PIX',
    'iugu_bank_slip' : 'BOLETO BANCARIO'}

for key, value in traducoes.items():
  df_faturas.loc[df_faturas.forma_pagamento==key,'forma_pagamento'] = value

### Definição da Região do Cliente com base no DDD

In [ ]:
# Possibilitando Análise Geoespacial dos dados
ddd_estados = {
    11: 'SP', 12: 'SP', 13: 'SP', 14: 'SP', 15: 'SP', 16: 'SP', 17: 'SP', 18: 'SP', 19: 'SP',
    21: 'RJ', 22: 'RJ', 24: 'RJ',
    27: 'ES',
    31: 'MG', 32: 'MG', 33: 'MG', 34: 'MG', 35: 'MG', 37: 'MG', 38: 'MG',
    41: 'PR', 42: 'PR', 43: 'PR', 44: 'PR', 45: 'PR', 46: 'PR',
    47: 'SC', 48: 'SC', 49: 'SC',
    51: 'RS', 53: 'RS',
    61: 'DF',
    62: 'GO', 64: 'GO',
    63: 'TO',
    65: 'MT',
    67: 'MS',
    68: 'AC',
    69: 'RO',
    71: 'BA', 73: 'BA', 74: 'BA', 75: 'BA', 77: 'BA',
    79: 'SE',
    81: 'PE', 87: 'PE',
    82: 'AL',
    83: 'PB',
    84: 'RN',
    85: 'CE',
    86: 'PI',
    88: 'CE',
    89: 'PI',
    91: 'PA', 93: 'PA', 94: 'PA',
    92: 'AM',
    95: 'RR',
    96: 'AP',
    97: 'AM',
    98: 'MA', 99: 'MA'
}

In [ ]:
df_faturas['ddd'] = df_faturas['ddd'].apply(lambda x: 0 if x==None else int(x))

In [ ]:
df_faturas['estado'] = df_faturas['ddd'].map(ddd_estados)

In [ ]:
regioes = {
    'AC': 'Norte','AM': 'Norte','AP': 'Norte','PA': 'Norte','RO': 'Norte','RR': 'Norte','TO': 'Norte',
    'AL': 'Nordeste','BA': 'Nordeste','CE': 'Nordeste','MA': 'Nordeste','PB': 'Nordeste','PE': 'Nordeste','PI': 'Nordeste','RN': 'Nordeste','SE': 'Nordeste',
    'DF': 'Centro-Oeste','GO': 'Centro-Oeste','MT': 'Centro-Oeste','MS': 'Centro-Oeste',
    'ES': 'Sudeste','MG': 'Sudeste','RJ': 'Sudeste','SP': 'Sudeste',
    'PR': 'Sul','RS': 'Sul','SC': 'Sul'}

In [ ]:
df_faturas['regiao'] = df_faturas['estado'].map(regioes)

### Identificação e Remoção da equipe soulcode da base de clientes

In [ ]:
df_faturas['dominio'] = df_faturas['email'].apply(lambda x: x.split('@')[1])

In [ ]:
df_faturas['dominio'].unique()

In [ ]:
df_soulcode = df_faturas.loc[(df_faturas['dominio'] == 'soulcode.com') |
                                        (df_faturas['dominio'] == 'teste.com') |
                                        (df_faturas['dominio'] == 'teste.com.br')
                                         ]
df_soulcode

In [ ]:
df_soulcode.groupby('dominio').size()

In [ ]:
df_faturas.info()

In [ ]:
for dominio in list(df_soulcode['dominio'].unique()):
  df_faturas = df_faturas.loc[df_faturas['dominio'] != dominio]

In [ ]:
df_faturas.info()

## Pré Análise e Insights Iniciais

In [ ]:
df_faturas.groupby(['status','a_pagar_com']).agg(quantidade=('status','size'),
                                                         media_parcelas=('max_parcelas','mean'),
                                                         total_taxas=('imposto_pago_centavos','sum'),
                                                         total_pago=('total_centavos','sum'),
                                                         total_reembolsado=('reembolsado_centavos','sum')
                                                          )

### Faturas por Status

In [ ]:
df_faturas['status'].value_counts()

In [ ]:
df_faturas['status'].value_counts(normalize=True) * 100

In [ ]:
df_faturas['status'].value_counts().plot.bar()

### Ticket Médio

In [ ]:
df_faturas['total_centavos'].hist()

### Forma de Pagamento mais utilizada

In [ ]:
df_faturas['forma_pagamento'].value_counts()

In [ ]:
df_faturas['forma_pagamento'].value_counts(normalize=True) * 100

In [ ]:
df_faturas['forma_pagamento'].value_counts().plot.pie(autopct='%1.1f%%')

### Análise de Correlação entre os valores da base

In [ ]:
cols_eda = ['total_centavos', 'pago_centavos', 'disconto_centavos', 'imposto_pago_centavos']

In [ ]:
df_heatmap = df_faturas[cols_eda]
df_pairplot = df_faturas[cols_eda]

In [ ]:
sns.heatmap(df_heatmap.corr(), cmap="YlGnBu", annot=True)

In [ ]:
sns.pairplot(df_pairplot)

## Conclusão da Pré Análise

- A maioria das faturas pagas foi através de cartão de crédito
- A quantidade de faturas canceladas é significativa(quase 15%)
- A maior parte das faturas expiradas são referentes a boletos bancários (mais de 60% dos boletos expirados)
- Em termos de valores, as faturas pagas por cartão de crédito representam a maior parte do total pago
- Apenas uma pequena porcentagem das faturas foi paga parcialmente
- A maioria das faturas não possui taxas
- As taxas cobradas para o pagamento por cartão de crédito são significativas em relação aos outros métodos de pagamento   
- Das faturas parceladas, a maioria é dividida em 12 parcelas   

## Validação dos dados com o SCHEMA

In [ ]:
df_faturas.replace(to_replace="None", value=np.NaN, inplace=True)

In [ ]:
df_faturas['criado_em'] = df_faturas['criado_em'].astype('datetime64[ns]')
df_faturas['atualizado_em'] = df_faturas['atualizado_em'].astype('datetime64[ns]')
df_faturas['pago_em'] = df_faturas['pago_em'].astype('datetime64[ns]')
df_faturas['expirado_em'] = df_faturas['expirado_em'].astype('datetime64[ns]')
df_faturas['protestado_em'] = df_faturas['protestado_em'].astype('datetime64[ns]')
df_faturas['cancelado_em'] = df_faturas['cancelado_em'].astype('datetime64[ns]')
df_faturas['chargeback_em'] = df_faturas['chargeback_em'].astype('datetime64[ns]')
df_faturas['data_ocorrencia'] = df_faturas['data_ocorrencia'].astype('datetime64[ns]')

In [ ]:
# Como agora já temos informação de Estado e Região o ddd não é mais necessário
df_faturas = df_faturas.drop('ddd', axis=1)

In [ ]:
'''
Abaixo foi constatada uma fatura que consta com o cliente id nulo, o que não deveria ocorrer.
Como os dados dessas faturas estavam todas como none, o status de pagamento pendende e é apenas uma
ocorrência, ela foi retirada dos dados.
'''

In [ ]:
df_faturas['cliente_id'].isna().any()

In [ ]:
df_faturas['cliente_id'].isna().sum()

In [ ]:
df_faturas.loc[df_faturas['cliente_id'].isnull()]

In [ ]:
df_faturas.drop(df_faturas.loc[df_faturas['cliente_id'].isnull()].index, inplace=True)

In [ ]:
df_faturas['cliente_id'].isna().any()

In [ ]:
# Ajustando tipos para o pandera
df_faturas["email_devido_ignorado"] = df_faturas["email_devido_ignorado"].map({True: "SIM", False: "NAO"})
df_faturas['reembolsavel'] = df_faturas['reembolsavel'].fillna(False).map({True: "SIM", False: "NAO"})

In [ ]:
'''
Essas faturas abaixo foram constatadas com a assinatura id nula, o que não deveria acontecer
Mas por parecer que são faturais de compras reais, e não testes, ter diferentes dados válidos,
e serem uma quantidade peqquena em relação ao total de faturas, elas foram mantidas.
'''

In [ ]:
df_faturas.loc[df_faturas['assinatura_id'].isnull()]

In [ ]:
#SCHEMA Faturas
schema_faturas = pa.DataFrameSchema({
    "fatura_id": pa.Column(pa.String),
    "cliente_id": pa.Column(pa.String, nullable=False),
    "assinatura_id": pa.Column(pa.String),
    "status": pa.Column(pa.String),
    "pago_centavos": pa.Column(pa.Float, nullable=True),
    "disconto_centavos": pa.Column(pa.Float, nullable=True),
    "multa_pagamento_atrasado": pa.Column(pa.Int),
    "imposto_pago_centavos": pa.Column(pa.Float, nullable=True),
    "total_pago": pa.Column(pa.String),
    "total_centavos": pa.Column(pa.Int),
    "items_total_centavos": pa.Column(pa.Int),
    "total_na_ocorrencia_centavos": pa.Column(pa.Int),
    "reembolsado_centavos": pa.Column(pa.Int),
    "total_reembolsado": pa.Column(pa.String),
    "parcelas": pa.Column(pa.Float, nullable=True),
    "max_parcelas": pa.Column(pa.Float, nullable=True),
    "total_pago_em_excesso": pa.Column(pa.String),
    "a_pagar_com": pa.Column(pa.String),
    "forma_pagamento": pa.Column(pa.String, nullable=True),
    "cartao_de_credito_marca": pa.Column(pa.String, nullable=True),
    "email_devido_ignorado": pa.Column(pa.String,pa.Check.str_length(3,3), nullable=True),
    "reembolsavel":  pa.Column(pa.String,pa.Check.str_length(3,3), nullable=True),
    "criado_em": pa.Column(pa.DateTime),
    "atualizado_em": pa.Column(pa.DateTime),
    "pago_em": pa.Column(pa.DateTime, nullable=True),
    "expirado_em": pa.Column(pa.DateTime, nullable=True),
    "protestado_em": pa.Column(pa.DateTime, nullable=True),
    "cancelado_em": pa.Column(pa.DateTime, nullable=True),
    "chargeback_em": pa.Column(pa.DateTime, nullable=True),
    "reembolsado_em": pa.Column(pa.String, nullable=True),
    "data_ocorrencia": pa.Column(pa.DateTime, nullable=True),
    "assinatura_id": pa.Column(pa.String, nullable=True),
    "estado": pa.Column(pa.String, nullable=True),
    "regiao": pa.Column(pa.String, nullable=True)
})

In [ ]:
df_faturas = schema_faturas.validate(df_faturas)

#Subcollection Items

##Extração

In [ ]:
df_items = df_invoices[['id','customer_id','items']]
df_items

In [ ]:
items_explode = df_items.explode('items')
df_items = pd.json_normalize(json.loads(items_explode.to_json(orient="records")))
df_items

In [ ]:
df_items.info()

##Tratamento

###Tradução

In [ ]:
df_faturas_itens = df_items.rename(columns={
    'id' : 'fatura_id',
    'customer_id': 'cliente_id',
    'items.id': 'item_id',
    'items.price' : 'preco',
    'items.description' : 'descricao',
    'items.quantity' : 'quantidade',
    'items.price_cents':'preco_centavos',
    'items.created_at' : 'criado_em',
    'items.updated_at':'atualizado_em'
})


In [ ]:
df_faturas_itens

In [ ]:
for c in df_faturas_itens.columns:
  print(f'coluna {c}: {list(df_faturas_itens[c].unique())}')

In [ ]:
df_faturas_itens['descricao'] = df_faturas_itens['descricao'].str.upper()

## Validação dos dados com o SCHEMA

In [ ]:
df_faturas_itens.replace(to_replace="None", value=np.NaN, inplace=True)

In [ ]:
df_faturas_itens['criado_em'] = df_faturas_itens['criado_em'].astype('datetime64[ns]')

In [ ]:
df_faturas_itens

In [ ]:
'''
Abaixo foi constatado um item de faturas com cliente id nulo, porém na descrição constava como teste
de multa, com isso foi tomada a decisão de dropar essa linha
'''

In [ ]:
df_faturas_itens.loc[df_faturas_itens['cliente_id'].isnull()]

In [ ]:
df_faturas_itens.drop(df_faturas_itens.loc[df_faturas_itens['cliente_id'].isnull()].index, inplace=True)

In [ ]:
faturas_itens_schema = pa.DataFrameSchema({
    "fatura_id": pa.Column(pa.String, nullable=False),
    "cliente_id": pa.Column(pa.String, nullable=False),
    "atualizado_em": pa.Column(pa.String, nullable=False),
    "preco_centavos": pa.Column(pa.Int, nullable=False),
    "descricao": pa.Column(pa.String, nullable=False),
    "criado_em": pa.Column(pa.DateTime, nullable=False),
    "item_id": pa.Column(pa.String, nullable=False),
    "quantidade": pa.Column(pa.Int, nullable=False),
    "preco": pa.Column(pa.String, nullable=False),
})

In [ ]:
df_faturas_itens = faturas_itens_schema.validate(df_faturas_itens)

#Collection CancellationOrder

##Extração

In [ ]:
db = firestore.client()

docs = db.collection('cancellationOrder').stream()

data = []
for doc in docs:
    data.append(doc.to_dict())

df_cancellationOrder = pd.DataFrame(data)

In [ ]:
df_cancellationOrder.info()

In [ ]:
df_cancellationOrder

##Tratamento

###Remover colunas nulas

In [ ]:
colunas_cancellationOrder = list(df_cancellationOrder.columns)

In [ ]:
#verificando existência de colunas nulas

colunas_nulas_cancellationOrder = []
total_nulos_cancellationOrder = 0

for c in colunas_cancellationOrder:
  if df_cancellationOrder[c].count() == 0:
    colunas_nulas_cancellationOrder.append(c)
    print(c)
    total_nulos_cancellationOrder += 1
print(f'Total de colunas nulas: {total_nulos_cancellationOrder}')

###Identificação das colunas principais

In [ ]:
df_ordem_cancelamento = df_cancellationOrder[['id', 'customer_id', 'invoice_id','customer_email', 'situation', 'sale_type',\
                                              'created_at', 'updated_at', 'refund_cents', 'fees_cents']]
df_ordem_cancelamento.info()

###Tradução

In [ ]:
df_ordem_cancelamento = df_ordem_cancelamento.rename(columns={
    'id' : 'ordem_cancelamento_id',
    'customer_id': 'cliente_id',
    'invoice_id': 'fatura_id',
    'customer_email': 'cliente_email',
    'situation': 'situacao',
    'sale_type': 'tipo_venda',
    'created_at' : 'criado_em',
    'updated_at' : 'atualizado_em',
    'refund_cents' : 'reembolso_centavos',
    'fees_cents' : 'taxas_centavos'
})

In [ ]:
for c in df_ordem_cancelamento.columns:
  print(f'coluna {c}: {list(df_ordem_cancelamento[c].unique())}')

In [ ]:
df_ordem_cancelamento['tipo_venda'] = df_ordem_cancelamento['tipo_venda'].str.upper()

In [ ]:
traducoes = {
    'pending':'PENDENTE',
    'complete':'COMPLETO',
    'error':'ERRO'
    }

for key, value in traducoes.items():
  df_ordem_cancelamento.loc[df_ordem_cancelamento.situacao==key,'situacao'] = value

### Identificação e Remoção da equipe soulcode da base de clientes

In [ ]:
df_ordem_cancelamento['dominio'] = df_ordem_cancelamento['cliente_email'].apply(lambda x: x.split('@')[1])

In [ ]:
df_ordem_cancelamento['dominio'].unique()

In [ ]:
df_soulcode = df_ordem_cancelamento.loc[(df_ordem_cancelamento['dominio'] == 'soulcode.com') |
                                        (df_ordem_cancelamento['dominio'] == 'teste.com') |
                                        (df_ordem_cancelamento['dominio'] == 'teste.com.br')
                                         ]
df_soulcode

In [ ]:
df_soulcode.groupby('dominio').size()

In [ ]:
df_ordem_cancelamento.info()

In [ ]:
for dominio in list(df_soulcode['dominio'].unique()):
  df_ordem_cancelamento = df_ordem_cancelamento.loc[df_ordem_cancelamento['dominio'] != dominio]

In [ ]:
df_ordem_cancelamento.info()

## Validação dos dados com o SCHEMA

In [ ]:
df_ordem_cancelamento.info()

In [ ]:
df_ordem_cancelamento['criado_em'] = df_ordem_cancelamento['criado_em'].astype('datetime64[ns]')
df_ordem_cancelamento['atualizado_em'] = df_ordem_cancelamento['atualizado_em'].astype('datetime64[ns]')

In [ ]:
schema_ordem_cancelamento = pa.DataFrameSchema({
"ordem_cancelamento_id": pa.Column(pa.String, nullable=False),
"cliente_id": pa.Column(pa.String, nullable=False),
"fatura_id": pa.Column(pa.String, nullable=False),
"situacao": pa.Column(pa.String, nullable=False),
"tipo_venda": pa.Column(pa.String, nullable=False),
"criado_em": pa.Column(pa.DateTime, nullable=False),
"atualizado_em": pa.Column(pa.DateTime, nullable=False),
"reembolso_centavos": pa.Column(pa.Int, nullable=False),
"taxas_centavos": pa.Column(pa.Int, nullable=False)
})

In [ ]:
df_ordem_cancelamento = schema_ordem_cancelamento.validate(df_ordem_cancelamento)

#Collection InvoiceExpiredData

##Extração

In [ ]:
db = firestore.client()

docs = db.collection('invoiceExpiredData').stream()

data = []
for doc in docs:
    data.append(doc.to_dict())

df_invoiceExpiredData = pd.DataFrame(data)

In [ ]:
df_invoiceExpiredData.info()

In [ ]:
df_invoiceExpiredData

##Tratamento

###Identificação das colunas principais

In [ ]:
df_faturas_expiradas = df_invoiceExpiredData[['id',
                                              'customer_id',
                                              'subscription_id',
                                              'email',
                                              'phone_prefix',
                                              'contract',
                                              'coupon_description',
                                              'paymentMethod',
                                              'invoice_total_cents',
                                              'plan_cents',
                                              'plan_paid_cents',
                                              'value_remaining_cents',
                                              'purchase_date',
                                              'due_date',
                                              'expired_days']]

###Tradução

In [ ]:
df_faturas_expiradas = df_faturas_expiradas.rename(columns={
    'id' : 'fatura_expirada_id',
    'customer_id': 'cliente_id',
    'subscription_id': 'assinatura_id',
    'phone_prefix': 'ddd',
    'contract': 'contrato',
    'coupon_description' : 'cupom_descricao',
    'paymentMethod' : 'forma_pagamento',
    'invoice_total_cents' : 'fatura_total_centavos',
    'plan_cents' : 'plano_centavos',
    'plan_paid_cents': 'plano_pago_centavos',
    'value_remaining_cents' : 'valor_faltante_centavos',
    'purchase_date' : 'data_compra',
    'due_date' : 'data_vencimento',
    'expired_days' : 'dias_expirados'
})


In [ ]:
for c in df_faturas_expiradas.columns:
  print(f'coluna {c}: {list(df_faturas_expiradas[c].unique())}')

In [ ]:
df_faturas_expiradas['cupom_descricao'] = df_faturas_expiradas['cupom_descricao'].str.upper()
df_faturas_expiradas['contrato'] = df_faturas_expiradas['contrato'].str.upper()

In [ ]:
traducoes = {
    'credit_card' : 'CARTAO DE CREDITO',
    'pix' : 'PIX',
    'bank_slip' : 'BOLETO BANCARIO'}

for key, value in traducoes.items():
  df_faturas_expiradas.loc[df_faturas_expiradas.forma_pagamento==key,'forma_pagamento'] = value

In [ ]:
df_faturas_expiradas.info()

### Identificação e Remoção da equipe soulcode da base de clientes

In [ ]:
df_faturas_expiradas['dominio'] = df_faturas_expiradas['email'].apply(lambda x: x.split('@')[1])

In [ ]:
df_faturas_expiradas['dominio'].unique()

In [ ]:
df_soulcode = df_faturas_expiradas.loc[(df_faturas_expiradas['dominio'] == 'soulcode.com') |
                                        (df_faturas_expiradas['dominio'] == 'teste.com') |
                                        (df_faturas_expiradas['dominio'] == 'teste.com.br')
                                         ]
df_soulcode

In [ ]:
df_soulcode.groupby('dominio').size()

In [ ]:
df_faturas_expiradas.info()

In [ ]:
for dominio in list(df_soulcode['dominio'].unique()):
  df_faturas_expiradas = df_faturas_expiradas.loc[df_faturas_expiradas['dominio'] != dominio]

In [ ]:
df_faturas_expiradas.info()

## Validação dos dados com o SCHEMA

In [ ]:
df_faturas_expiradas['data_compra'] = df_faturas_expiradas['data_compra'].astype('datetime64[ns]')
df_faturas_expiradas['data_vencimento'] = df_faturas_expiradas['data_vencimento'].astype('datetime64[ns]')

In [ ]:
schema_faturas_expiradas = pa.DataFrameSchema({
    "fatura_expirada_id": pa.Column(pa.String, nullable=False),
    "cliente_id": pa.Column(pa.String, nullable=False),
    "assinatura_id": pa.Column(pa.String, nullable=False),
    "ddd": pa.Column(pa.String, nullable=False),
    "contrato": pa.Column(pa.String, nullable=False),
    "cupom_descricao": pa.Column(pa.String, nullable=False),
    "forma_pagamento": pa.Column(pa.String, nullable=False),
    "fatura_total_centavos": pa.Column(pa.Int, nullable=False),
    "plano_centavos": pa.Column(pa.Int, nullable=False),
    "plano_pago_centavos": pa.Column(pa.Int, nullable=False),
    "valor_faltante_centavos": pa.Column(pa.Int, nullable=False),
    "data_compra": pa.Column(pa.DateTime, nullable=False),
    "data_vencimento": pa.Column(pa.DateTime, nullable=False),
    "dias_expirados": pa.Column(pa.Int, nullable=False)
})

In [ ]:
df_faturas_expiradas = schema_faturas_expiradas.validate(df_faturas_expiradas)

#Collection Receipts

## Extração

In [ ]:
db = firestore.client()

docs = db.collection('receipts').stream()

data = []
for doc in docs:
    data.append(doc.to_dict())

df_receipts = pd.DataFrame(data)

In [ ]:
df_receipts.info()

In [ ]:
df_receipts

## Tratamento

### Remover colunas nulas

In [ ]:
colunas_receipts = list(df_receipts.columns)

In [ ]:
print(len(colunas_receipts))

In [ ]:
colunas_nulas_df_receipts = []
total_nulos_df_receipts = 0

for c in list(df_receipts.columns):
  if df_receipts[c].count() == 0:
    colunas_nulas_df_receipts.append(c)
    print(c)
    total_nulos_df_receipts += 1
print(f'Total de colunas nulas: {total_nulos_df_receipts}')

In [ ]:
for c in colunas_receipts:
  if c in colunas_nulas_df_receipts:
    colunas_receipts.remove(c)

print(len(colunas_receipts))

###Remover subcollections

In [ ]:
colunas_subcollection_receipts = ['nfe','produto', 'cliente']
print(f'Qtd de Subcollections: {len(colunas_subcollection_receipts)}')

In [ ]:
for c in colunas_receipts:
  if c in colunas_subcollection_receipts:
    colunas_receipts.remove(c)

print(len(colunas_receipts))

In [ ]:
colunas_receipts

###Identificação das colunas principais

In [ ]:
colunas_receipts_principais = ['id', 'dataVenda','meioDePagamento', 'situacao' , 'valorTotal' , 'produto' ]

In [ ]:
df_receipts_final = df_receipts[colunas_receipts_principais]

In [ ]:
df_receipts_final

### Tradução

In [ ]:
df_recibos = df_receipts_final.rename(columns={
    'id':'recibo_id',
    'dataVenda':'vendido_em',
    'meioDePagamento':'forma_pagamento',
    'situacao':'situacao',
    'valorTotal':'valorTotal',
    'produto':'produto'
    })

In [ ]:
df_recibos

### Extração da subcollection Produtos

In [ ]:
'''
Aqui decidimos unir a subcoleção com o id coleção principal para ser possível criar
um relacionamento, e então concatenar as informações úteis da subcoleção a coleção principal
'''

In [ ]:
df_recibos_produto = df_recibos[['recibo_id','produto']]

In [ ]:
df_recibos = df_recibos.drop('produto', axis=1)

In [ ]:
df_recibos

In [ ]:
df_recibos_produto

In [ ]:
df_recibos_produto['produto'].apply(pd.Series)

In [ ]:
df_recibos_produto_final = pd.concat([df_recibos_produto.drop(['produto'], axis=1), df_recibos_produto['produto'].apply(pd.Series)], axis=1)

In [ ]:
df_recibos_produto_final

In [ ]:
df_recibos_produto = df_recibos_produto_final.rename(columns={
    'recibo_id':'produto_recibo_id',
    'idExterno':'produto_id_externo',
    'id':'produto_id',
    'descricao':'descricao'
    })

In [ ]:
df_recibos_produto

In [ ]:
df_recibos_e_produtos = pd.concat([df_recibos, df_recibos_produto], axis=1)

In [ ]:
df_recibos_e_produtos

In [ ]:
df_recibos_e_produtos['recibo_id'].nunique()

## Validação dos dados com o SCHEMA

In [ ]:
df_recibos_e_produtos.replace(to_replace="None", value=np.NaN, inplace=True)

In [ ]:
df_recibos_e_produtos

In [ ]:
df_recibos_e_produtos.info()

In [ ]:
df_recibos_e_produtos['vendido_em'] = df_recibos_e_produtos['vendido_em'].astype('datetime64[ns]')

In [ ]:
df_recibos_e_produtos['situacao'].unique()

In [ ]:
schema_recibos = pa.DataFrameSchema({
"recibo_id": pa.Column(pa.String, nullable=False),
"vendido_em": pa.Column(pa.DateTime, nullable=False),
"forma_pagamento": pa.Column(pa.String, nullable=False),
"situacao": pa.Column(pa.String, nullable=False),
"valorTotal": pa.Column(pa.Float, nullable=False),
"produto_recibo_id": pa.Column(pa.String, nullable=False),
"produto_id": pa.Column(pa.String, nullable=False),
"produto_id_externo": pa.Column(pa.String, nullable=False),
"descricao": pa.Column(pa.String, nullable=False),
})

In [ ]:
df_recibos_e_produtos = schema_recibos.validate(df_recibos_e_produtos)

#Exportação dos Dados Tratados

In [ ]:
#Cria um dicionário com os dataframes validados para carregá-los de uma vez
dfs_validados = {'df_assinaturas':df_assinaturas,
                 'df_carrinho':df_carrinho,
                 'df_faturas':df_faturas,
                 'df_faturas_itens':df_faturas_itens,
                 'df_ordem_cancelamento':df_ordem_cancelamento,
                 'df_faturas_expiradas':df_faturas_expiradas,
                 'df_recibos_e_produtos':df_recibos_e_produtos
}


##Bucket

In [ ]:
#Transforma cada dataframe em um arquivo .csv e os envia ao bucket utilizando o caminho gsutil

for nome, df in dfs_validados.items():
  df.to_csv(f'gs://ad1-dashboards-b2c-tratados/{nome}.csv')



##MongoDB

In [ ]:
#Transformação das colunas do tipo data dos dfs para objeto. Ação necessária para contornar
#incompatibilidade do pymongo com os dados NaT

colunas_data = ['data_compra',
                'data_vencimento',
                'criado_em',
                'atualizado_em',
                'pago_em',
                'expirado_em',
                'protestado_em',
                'cancelado_em',
                'chargeback_em',
                'data_ocorrencia',
                'ultima_atualizacao',
                'expira_em',
                'data']

In [ ]:
df_validados_mongo = dict()

for nome, df in dfs_validados.items():
  for coluna in colunas_data:
    if coluna in list(df.columns):
      df[coluna] = df[coluna].astype(object).where(df[coluna].notnull(), None)

  df_validados_mongo[nome] = df

In [ ]:
#Transforma cada dataframe em um dicionário de registros e os insere em uma coleção do MongoDB


for i, (key, value) in enumerate(df_validados_mongo.items()):

  nome_db = f'{i}'
  db = client_mongo[nome_db]

  colecao = db[key]

  df_dict = value.to_dict("records")

  colecao.insert_many(df_dict)



# EDA PySpark

## Faturas

In [ ]:
df_faturas_spark = spark.createDataFrame(df_faturas)

In [ ]:
df_faturas_spark.show()

## Fatura_Itens

In [ ]:
df_faturas_itens_spark = spark.createDataFrame(df_faturas_itens)

In [ ]:
df_faturas_itens_spark.show()

## Faturas Expiradas

In [ ]:
df_faturas_expiradas_spark = spark.createDataFrame(df_faturas_expiradas)

In [ ]:
df_faturas_expiradas_spark.show()

## Assinaturas

In [ ]:
df_subscriptions_spark = spark.createDataFrame(df_subscriptions_geral)

In [ ]:
df_subscriptions_spark.show()

## ShoppingCart

In [ ]:
df_shoppingCart_spark = spark.createDataFrame(df_shoppingCart_final)

In [ ]:
df_shoppingCart_spark.show()

## Recibos e Produtos

In [ ]:
df_recibos_e_produtos_spark = spark.createDataFrame(df_recibos_e_produtos)

In [ ]:
df_recibos_e_produtos_spark.show()

## Ordem de Cancelamento

In [ ]:
df_ordem_cancelamento_spark = spark.createDataFrame(df_ordem_cancelamento)

In [ ]:
df_ordem_cancelamento_spark.show()

# Dashboard

[Trabalho Final - Dashboard Assinatura Tech da SoulCode Academy](https://lookerstudio.google.com/reporting/0bdc0001-ec57-4964-9a53-d35e35ff189e/page/rIJPD)